# Caculate the energy balance

In [1]:
# Import the needed packages

import numpy as np
import pandas as pd
import timeit

In [2]:
# Pick whatever file you want
df_exp = pd.read_csv('results/outcome_cd')
#df_exp = pd.read_csv('results/outcome_cm')
#df_exp = pd.read_csv('results/outcome_wy')
#df_exp = pd.read_csv('results/outcome_bc')

df_exp['point'] = df_exp['x'].astype(str) + ', ' + df_exp['y'].astype(str)

In [3]:
# Open dataframe with hourly data
df_hour = pd.read_csv('data/residential_and_wh_hourly_innercity.csv')

C:\Users\sarah\AppData\Local\Temp\ipykernel_16156\4108588489.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hour = pd.read_csv('data/residential_and_wh_hourly_innercity.csv')


In [4]:
# Add the identified clusters as column to the hourly dataframe

df_hour["error"] = 1
df_hour["clusters"] = np.nan

values = df_exp.point.unique()

for i in range(len(df_hour)):
    point = df_hour.loc[i, "point"]
    x = df_hour.loc[i, "x"]
    y = df_hour.loc[i, "y"]
    #if point in values:
    if str(x) + ', ' + str(y) in values:
        df_hour.loc[i, "error"] = 0
        place = np.where(df_exp["point"] == str(x) + ', ' + str(y))
        #try:
        index = place[0][0].item()
        cluster = df_exp.iloc[index]["clusters"]
        df_hour.loc[i, "clusters"] = cluster
        #except:
         #   index = place[0][0].item()
          #  cluster = df_exp.iloc[index]["clusters"]
           # df_hour.loc[i, "clusters"] = cluster
            #continue
    else:
        print(point)

In [5]:
df_exp.clusters.unique()

array([117., 135., 101., 140., 142., 145.,  78., 141., 134.,  80.,  65.,
        60., 139., 147., 144., 146.,  53., 106.,  76.,  35.,  45., 105.])

Choose if you want to calculate the hourly lack of supply or the hourly disbalance

In [6]:
# SUPPLY SHORTAGE
start = timeit.default_timer()

energy_clusters = []
energy = 0
month = [31,59,90,120,151,181,212,243,273,304,334,365]
list_with_values = []

# For every cluster
for cluster in df_hour.clusters.unique():
    # Calculate the supply lack over the year
    energy_month = 0
    m = 0
    list_in_list = []
    # For each hour in a year
    for hour in range(8760):
        if df_hour[str(hour)].loc[df_hour.clusters == cluster].sum() < 0:
            energy_month += df_hour[str(hour)].loc[df_hour.clusters == cluster].sum()
        if hour == (month[m]*24-1):
            m += 1
            list_in_list.append(energy_month)
            energy_month = 0
    list_with_values.append(list_in_list)
            
        
    energy += sum(list_in_list)
    
    print('this year has a supply shortage of', sum(list_in_list), 'in cluster: ', cluster)
    
print('there is total a supply shortage of', energy, 'in all clusters')

list_with_keys = df_hour.clusters.unique()
data = dict(zip(list_with_keys, list_with_values))

stop = timeit.default_timer()

print('Time: ', stop - start)

this year has a supply shortage of -6421414.583281044 in cluster:  117.0
this year has a supply shortage of -3543543.04608518 in cluster:  135.0
this year has a supply shortage of -4326927.40941592 in cluster:  101.0
this year has a supply shortage of -16386558.640615152 in cluster:  140.0
this year has a supply shortage of -5429403.762610285 in cluster:  142.0
this year has a supply shortage of -4671660.261330447 in cluster:  145.0
this year has a supply shortage of -731073.0481715108 in cluster:  78.0
this year has a supply shortage of -8346407.630086092 in cluster:  141.0
this year has a supply shortage of -5033341.884516681 in cluster:  134.0
this year has a supply shortage of -1395067.6642861273 in cluster:  80.0
this year has a supply shortage of -4957688.51335245 in cluster:  65.0
this year has a supply shortage of -2825690.8113346817 in cluster:  60.0
this year has a supply shortage of -630058.7498920878 in cluster:  139.0
this year has a supply shortage of -8608340.092597913 i

In [7]:
df = pd.DataFrame(data=data)

In [8]:
df.reset_index(inplace=True)
df.rename(columns = {'index':'Month'}, inplace = True)
df['Month'] = df['Month'] + 1
df = df.set_index('Month')
df.to_excel('results/experiments_cd.xlsx')

In [9]:
df

,117.0,135.0,101.0,140.0,142.0,145.0,78.0,141.0,134.0,80.0,...,139.0,147.0,144.0,146.0,53.0,106.0,76.0,35.0,45.0,105.0
Month,,,,,,,,,,,,,,,,,,,,,
1,-1.146067e+06,-655562.787284,-809396.467660,-2.758423e+06,-980383.301428,-794749.835151,-157592.149999,-1.354513e+06,-1.004997e+06,-299992.256279,...,-129772.437186,-1.421109e+06,-1.418202e+06,-629789.767718,-1.552975e+06,-883472.717938,-131125.057310,-612556.660773,-941594.493841,-957526.532431
2,-1.170936e+06,-646886.286062,-795080.703995,-2.639359e+06,-948415.622307,-754850.212220,-164758.315103,-1.259202e+06,-9.946387e+05,-299938.593587,...,-134350.806432,-1.325136e+06,-1.324055e+06,-588197.136745,-1.512773e+06,-802910.496300,-140006.230444,-587595.879281,-884607.053891,-885558.393917
3,-1.026316e+06,-574423.956047,-708595.550572,-2.384172e+06,-820150.006352,-673151.532948,-114043.031143,-1.131591e+06,-7.927512e+05,-222750.295317,...,-101811.100753,-1.186264e+06,-1.214839e+06,-515166.979323,-1.347375e+06,-717583.055815,-106837.518834,-533093.656679,-812359.454425,-760340.079792
4,-4.920298e+05,-256202.901214,-297334.438788,-1.320643e+06,-401592.251453,-368674.467756,-22644.269216,-6.609524e+05,-3.044038e+05,-58131.089298,...,-26110.389078,-6.797015e+05,-7.493734e+05,-277822.614172,-7.012773e+05,-431623.135217,-26086.401743,-297615.708963,-493745.819237,-394037.657991
5,-1.845055e+05,-86104.667524,-91099.661096,-6.324814e+05,-168791.258484,-176262.804288,-1900.858472,-3.557102e+05,-9.119495e+04,-11574.471056,...,-3125.254380,-3.532172e+05,-4.363711e+05,-132394.032124,-3.322645e+05,-255414.986870,-3669.490765,-146015.533561,-284228.949976,-183186.210020
6,-6.389851e+04,-29221.872060,-34790.813669,-3.149847e+05,-68286.205366,-89360.797484,0.000000,-2.110500e+05,-9.193892e+03,0.000000,...,0.000000,-2.007882e+05,-2.770102e+05,-66788.390094,-1.704684e+05,-169903.645774,0.000000,-75628.572157,-177082.327641,-88956.959487
7,-4.079329e+04,-17929.629656,-21617.322434,-2.652343e+05,-53948.190621,-79223.630279,0.000000,-1.975341e+05,-4.986654e+03,0.000000,...,0.000000,-1.848981e+05,-2.671912e+05,-60046.236455,-1.399818e+05,-160979.241277,0.000000,-63466.812133,-172013.578442,-78746.367187
8,-4.417587e+04,-18130.369812,-21862.359409,-2.670437e+05,-52844.820507,-77087.284151,0.000000,-1.909031e+05,-4.920957e+03,0.000000,...,0.000000,-1.787210e+05,-2.600739e+05,-56965.966168,-1.386043e+05,-154731.578160,0.000000,-64111.604271,-168628.330257,-72152.841144
9,-3.675416e+04,-13153.086250,-13748.894687,-2.670520e+05,-48835.671287,-76278.206969,0.000000,-1.961059e+05,-4.378252e+03,0.000000,...,0.000000,-1.826968e+05,-2.733113e+05,-55696.557397,-1.221030e+05,-155681.457709,0.000000,-64819.134818,-176089.021333,-66348.896516
